<a href="https://colab.research.google.com/github/itsmerajesh4990/AIpracticeandtraining/blob/main/ENEC_ticket_crewai_new.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
import os
from google.colab import userdata

# Retrieve API key from Colab secrets and set it in environment variables
os.environ["OPENAI_API_KEY"] = userdata.get('openai')

In [10]:
import json
import os

# 1. Define the Ticket Data (The "Dummy Data")
tickets = [
    {
        "ticket_id": "TIC-GEN-001", # Placeholder ticket_id
        "metadata": {
            "created_at": "2023-10-26T10:00:00Z",
            "sla_tier": "gold"
        },
        "customer_profile": {
            "name": "Alice Wonderland",
            "role": "IT Support",
            "history": { "sentiment_score": 0.1 }
        },
        "issue_content": {
            "subject": "General system query",
            "priority_level": "medium",
            "raw_description": "Testing the ticket system functionality."
        },
        "technical_context": {
            "error_codes": [], # Corrected empty list
            "logs": "System check initiated."
        },
        "lifecycle": { "status": "open", "resolution_notes": "" }
    },
    {
        "ticket_id": "TIC-IAM-002",
        "metadata": {
            "created_at": "2023-10-27T08:45:00Z",
            "sla_tier": "platinum"
        },
        "customer_profile": {
            "name": "Victor Chen",
            "role": "CFO",
            "history": { "sentiment_score": -0.5 }
        },
        "issue_content": {
            "subject": "URGENT: Cannot approve wires",
            "priority_level": "critical",
            "raw_description": "My token isn't working for the ERP system. I need this fixed immediately."
        },
        "technical_context": {
            "error_codes": [], # Corrected empty list
            "logs": "MFA Challenge Initiated. No response."
        },
        "lifecycle": { "status": "open", "resolution_notes": "" }
    },
    {
        "ticket_id": "TIC-HW-105",
        "metadata": {
            "created_at": "2023-10-27T09:30:00Z",
            "sla_tier": "gold"
        },
        "customer_profile": {
            "name": "Elena Fisher",
            "role": "Journalist",
            "history": { "sentiment_score": -0.8 }
        },
        "issue_content": {
            "subject": "Blue Screen of Death",
            "priority_level": "high",
            "raw_description": "Laptop keeps crashing with blue screen."
        },
        "technical_context": {
            "error_codes": [], # Corrected empty list
            "logs": "Caused by driver: nvlddmkm.sys"
        },
        "lifecycle": { "status": "open", "resolution_notes": "" }
    }
]

# 2. Define Knowledge Base Articles (The RAG Content)
kb_articles = {
    "KB_IAM_001_MFA_Sync.md": """
# Knowledge Base: MFA Token Issues
**Related Error Codes:** INVALID_TOKEN, MFA_TIMEOUT
## Symptom
User reports token rejected or push not arriving.
## Resolution Steps
1. **Time Drift:** Instruct user to sync time in Authenticator App settings.
2. **Network:** Toggle Wi-Fi off/on.
""",
    "KB_OS_WIN_004_BSOD.md": """
# Knowledge Base: Windows BSOD 0x0000000A
**Related Error Codes:** 0x0000000A, IRQL_NOT_LESS_OR_EQUAL
## Root Cause
Driver compatibility issue, often Nvidia (nvlddmkm.sys).
## Resolution Steps
1. Boot into Safe Mode.
2. Rollback the specific driver in Device Manager.
"""
}

def setup():
    # Write Tickets
    with open('tickets.json', 'w') as f:
        json.dump(tickets, f, indent=2)
    print("✅ Generated tickets.json")

    # Write Knowledge Base
    if not os.path.exists('knowledge_base'):
        os.makedirs('knowledge_base')

    for filename, content in kb_articles.items():
        with open(os.path.join('knowledge_base', filename), 'w') as f:
            f.write(content.strip())
    print("✅ Generated Knowledge Base articles")

if __name__ == "__main__":
    setup()


✅ Generated tickets.json
✅ Generated Knowledge Base articles


In [11]:
tickets

[{'ticket_id': 'TIC-GEN-001',
  'metadata': {'created_at': '2023-10-26T10:00:00Z', 'sla_tier': 'gold'},
  'customer_profile': {'name': 'Alice Wonderland',
   'role': 'IT Support',
   'history': {'sentiment_score': 0.1}},
  'issue_content': {'subject': 'General system query',
   'priority_level': 'medium',
   'raw_description': 'Testing the ticket system functionality.'},
  'technical_context': {'error_codes': [], 'logs': 'System check initiated.'},
  'lifecycle': {'status': 'open', 'resolution_notes': ''}},
 {'ticket_id': 'TIC-IAM-002',
  'metadata': {'created_at': '2023-10-27T08:45:00Z', 'sla_tier': 'platinum'},
  'customer_profile': {'name': 'Victor Chen',
   'role': 'CFO',
   'history': {'sentiment_score': -0.5}},
  'issue_content': {'subject': 'URGENT: Cannot approve wires',
   'priority_level': 'critical',
   'raw_description': "My token isn't working for the ERP system. I need this fixed immediately."},
  'technical_context': {'error_codes': [],
   'logs': 'MFA Challenge Initiate

In [12]:
!pip install crewai crewai-tools nest_asyncio

This code builds a fully autonomous support team simulation inside your notebook. Instead of just "chatting" with an AI, you are creating two distinct AI employees (Agents), giving them specific job descriptions, and providing them with a "virtual office" (the JSON and Markdown files) where they can do real work.

Here is the step-by-step breakdown of what is happening under the hood:

##1. The "Colab Fix" (nest_asyncio)
What it does: Google Colab (and Jupyter Notebooks) runs inside a Python "Event Loop" to manage its interactive cells. CrewAI also tries to start its own Event Loop to manage the agents. Normally, Python forbids running two loops at the same time. Why we need it: nest_asyncio.apply() patches Colab to allow these loops to run inside each other. Without this line, the code would crash with a RuntimeError before the agents even start.

##2. Creating the "Virtual Office" (Data Generation)
AI agents cannot hallucinate a database; they need files to interact with.

tickets.json: This file acts as your company's database. It contains the "problems" (e.g., the CFO's broken token, the blue screen crash). We purposely include different types of data (SLA tiers, error codes) to test if the agents can read them correctly.

knowledge_base/: We create a folder with Markdown (.md) files. This simulates a company Wiki or Confluence page. It contains the "answers" (e.g., how to fix MFA issues).

##3. Giving Agents "Hands" and "Eyes" (Tool Definitions)
By default, an AI model (like GPT-4) is just a brain in a jar. It can't read your files. We solve this by creating Tools.

TicketingSystemTool: This is a Python class that acts as an API.

When the agent says "I want to read the queue," this code actually opens tickets.json, reads the text, and sends it back to the agent.

When the agent says "I want to update status to resolved," this code writes that change into the JSON file.

SimpleRAGTool: This simulates a search engine. When the agent asks for "MFA help," this tool loops through the text files in the knowledge_base folder, counts matching keywords, and returns the most relevant file.

##4. Hiring the Staff (Agent Definitions)
We define two distinct personalities. This is called "Role-Based Agent Architecture."

The Planner (Support Triage Manager):

Goal: Look at the big picture.

Restriction: Notice that this agent does not have the rag_tool. It cannot look up technical answers. It is forced to focus only on prioritization (SLA, timestamps, urgency).

The Analyst (Technical Issue Analyst):

Goal: Solve the specific problem.

Capability: This agent has both tools. It takes the specific ticket assigned by the Planner, researches it, and pushes the fix.

##5. Defining the Workflow (Tasks)
We tell the crew exactly what to do, step-by-step.

Task 1 (Triage): "Read the queue and give me the ID of the most critical ticket."

Expected behavior: The Planner looks at the list, sees the "Platinum/Critical" tag on the CFO's ticket, and outputs TIC-IAM-002.

Task 2 (Resolve): "Take the ID from Task 1, research it, and fix it."

Expected behavior: The Analyst receives TIC-IAM-002. It reads the details, sees "MFA Token," searches the Knowledge Base, finds the "Time Sync" fix, and updates the ticket.

##6. The Execution (crew.kickoff)
When you run support_crew.kickoff(), the magic happens:

The Planner starts. It calls the TicketingTool, reads the JSON, thinks "The CFO is angry, that's priority #1," and passes the ID to the next agent.

The Analyst wakes up. It receives the ID. It calls the TicketingTool to get details. It realizes it doesn't know how to fix "MFA." It calls the RAGTool. It gets the answer. It calls the TicketingTool again to write the solution.

##Final Result:
 The script prints the final report, and if you open tickets.json in Colab's file viewer, you will see the status has actually changed to resolved.

In [13]:
# ----------------------------
# Imports
# ----------------------------
import json          # For reading/writing JSON (tickets.json, and printing ticket details)
import os            # For listing files in the knowledge_base directory

# CrewAI core building blocks:
from crewai import Agent, Task, Crew, Process

# BaseTool is the parent class used to define custom tools for agents
from crewai.tools import BaseTool

# Pydantic is used to define tool input schemas (validations + descriptions)
from pydantic import BaseModel, Field

# Typing utilities
from typing import Optional, Type


# ==========================================================
# --- TOOL DEFINITIONS ---
# ==========================================================

# ----------------------------
# 1) Ticket Tool Input Schema
# ----------------------------
class TicketActionInput(BaseModel):
    # "action" tells the tool what operation to perform
    action: str = Field(
        ...,
        description="Action to perform: 'read_queue', 'get_ticket', or 'update_status'."
    )

    # Ticket ID is only needed for "get_ticket" and "update_status"
    ticket_id: Optional[str] = Field(
        None,
        description="The ID of the ticket (e.g., TIC-IAM-002). Required for details/updates."
    )

    # new_status is only needed for update_status (e.g., resolved)
    new_status: Optional[str] = Field(
        None,
        description="New status (e.g., 'resolved'). Required for update_status."
    )

    # agent_notes are only needed for update_status (what you did to fix it)
    agent_notes: Optional[str] = Field(
        None,
        description="Notes to add to the ticket. Required for update_status."
    )


# ----------------------------
# 2) Ticketing System Tool
# ----------------------------
class TicketingSystemTool(BaseTool):
    # Tool metadata: name & description show up in the agent's tool list
    name: str = "Ticketing_System_Interface"
    description: str = (
        "Accesses the support ticket database. "
        "Use 'read_queue' to see open tickets, 'get_ticket' for details, "
        "and 'update_status' to close them."
    )

    # args_schema tells CrewAI: "these are the fields you must pass"
    # CrewAI will validate tool inputs using this schema.
    args_schema: Type = TicketActionInput

    # _run is what actually executes when the agent calls the tool.
    # NOTE: CrewAI tools can also be async in some setups; here it's sync.
    def _run(
        self,
        action: str,
        ticket_id: str = None,
        new_status: str = None,
        agent_notes: str = None
    ) -> str:

        # Hardcoded path to the "database" file
        file_path = "tickets.json"

        # Try to read tickets from tickets.json
        try:
            with open(file_path, 'r') as f:
                tickets = json.load(f)  # tickets should be a list[dict]
        except Exception as e:
            # If file missing or JSON invalid, return an error message string
            return f"Error accessing database: {str(e)}"

        # -------------
        # Action: read_queue
        # -------------
        if action == "read_queue":
            # Build a readable summary line for each OPEN ticket
            # It assumes a specific nested structure inside each ticket dict.
            # (If your JSON structure differs, you'll get KeyError.)
            return "\n".join([
                f"Ticket ID: {t['ticket_id']} | "
                f"Priority: {t['issue_content']['priority_level']} | "
                f"SLA: {t['metadata']['sla_tier']} | "
                f"Subject: {t['issue_content']['subject']}"
                for t in tickets
                if t['lifecycle']['status'] == 'open'
            ])

        # -------------
        # Action: get_ticket
        # -------------
        elif action == "get_ticket":
            # Search for the ticket by ticket_id and return the full ticket JSON
            for t in tickets:
                if t['ticket_id'] == ticket_id:
                    # Return prettified JSON for readability
                    return json.dumps(t, indent=2)
            return "Ticket not found."

        # -------------
        # Action: update_status
        # -------------
        elif action == "update_status":
            # Find ticket and update fields; then write the list back to file
            for t in tickets:
                if t['ticket_id'] == ticket_id:
                    # Update the ticket lifecycle fields
                    t['lifecycle']['status'] = new_status
                    t['lifecycle']['resolution_notes'] = agent_notes

                    # Persist changes back into tickets.json
                    with open(file_path, 'w') as f:
                        json.dump(tickets, f, indent=2)

                    return f"Ticket {ticket_id} updated successfully."
            return "Ticket not found."

        # -------------
        # If action is unknown, tool returns nothing currently (None).
        # In practice, it's better to return a helpful message.
        # -------------
        # (Not included in your original code, but good to know.)
        # else:
        #     return f"Unknown action: {action}"


# ----------------------------
# 3) RAG Tool Input Schema
# ----------------------------
class RAGInput(BaseModel):
    # The query that we will match against knowledge base articles
    query: str = Field(..., description="Keywords or error codes to search for.")


# ----------------------------
# 4) Simple RAG Tool
# ----------------------------
class SimpleRAGTool(BaseTool):
    name: str = "Knowledge_Base_RAG"
    description: str = (
        "Searches past solutions. Useful for finding technical fixes based on error codes or descriptions."
    )
    args_schema: Type = RAGInput

    def _run(self, query: str) -> str:
        # Directory containing markdown knowledge articles (*.md)
        kb_dir = "knowledge_base"

        # Track the best matching article content (simple scoring)
        best_match = ""
        highest_score = 0

        # Loop through all files in knowledge_base/
        for filename in os.listdir(kb_dir):
            # Only consider markdown files
            if filename.endswith(".md"):
                # Read file content
                with open(os.path.join(kb_dir, filename), 'r') as f:
                    content = f.read()

                    # VERY basic keyword scoring:
                    # score = number of query words found as substrings in content
                    score = sum(
                        1
                        for word in query.lower().split()
                        if word in content.lower()
                    )

                    # Keep track of the best-scoring document
                    if score > highest_score:
                        highest_score = score
                        best_match = content

        # If some match exists, return the article
        if highest_score > 0:
            return f"Found relevant article:\n{best_match}"

        # Otherwise no results
        return "No relevant articles found."


# ==========================================================
# --- AGENT & CREW SETUP ---
# ==========================================================

# Instantiate your tools so agents can use them
ticket_tool = TicketingSystemTool()
rag_tool = SimpleRAGTool()

# ----------------------------
# Agent 1: Planner / Triage Manager
# ----------------------------
planner = Agent(
    role="Support Triage Manager",
    goal="Prioritize the ticket queue based on SLA and Severity.",
    backstory=(
        "You are a veteran support manager. "
        "You prioritize High Value Customers (Platinum) and Critical issues above all else."
    ),
    tools=[ticket_tool],         # Planner can ONLY access the ticket system tool
    verbose=True,                # Prints reasoning/steps (depends on CrewAI verbosity)
    allow_delegation=False       # Prevents this agent from delegating to other agents
)

# ----------------------------
# Agent 2: Analyst / Troubleshooter
# ----------------------------
analyst = Agent(
    role="Technical Issue Analyst",
    goal="Diagnose the issue using the Knowledge Base and resolve the ticket.",
    backstory=(
        "You are a senior troubleshooter. "
        "You read the ticket details, search the knowledge base for error codes or keywords, "
        "and apply the fix."
    ),
    tools=[ticket_tool, rag_tool],  # Analyst can read/update tickets AND search KB
    verbose=True,
    allow_delegation=False
)

# ----------------------------
# Task 1: Triage
# ----------------------------
task_triage = Task(
    description=(
        "1. Use 'read_queue' to view tickets.\n"
        "2. Identify the most critical ticket (Platinum/Critical).\n"
        "3. Pass the Ticket ID to the next task."
    ),
    expected_output="The Ticket ID of the most critical ticket.",
    agent=planner
)

# ----------------------------
# Task 2: Resolve
# ----------------------------
task_resolve = Task(
    description=(
        "1. Get ticket details for the ID provided.\n"
        "2. Search Knowledge Base for symptoms.\n"
        "3. Update status to 'resolved' with notes."
    ),
    expected_output="Confirmation that the ticket is resolved.",
    agent=analyst
)

# ----------------------------
# Crew: orchestrates agents + tasks
# ----------------------------
support_crew = Crew(
    agents=[planner, analyst],          # The two agents participating
    tasks=[task_triage, task_resolve],  # Run triage then resolve
    process=Process.sequential          # Sequential means Task2 runs after Task1 output
)

# ==========================================================
# --- EXECUTION ---
# ==========================================================
print("### Starting Support Crew Simulation ###")

# kickoff() runs the workflow:
# - planner runs task_triage (using ticket_tool)
# - analyst runs task_resolve (using ticket_tool + rag_tool),
#   with context from Task1's output
result = support_crew.kickoff()

# Print final output
print("\n\n########################")
print("## Final Result ##")
print("########################\n")
print(result)


### Starting Support Crew Simulation ###


╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Support Triage Manager                                                                                  │
│                                                                                                                 │
│  Task: 1. Use 'read_queue' to view tickets.                                                                     │
│  2. Identify the most critical ticket (Platinum/Critical).                                                      │
│  3. Pass the Ticket ID to the next task.                                                                        │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────── 🔧 Agent Tool Execution ────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Support Triage Manager                                                                                  │
│                                                                                                                 │
│  Thought: Thought: I need to read the ticket queue first to get a list of all open tickets. Then I will         │
│  identify the most critical ticket based on the criteria of Platinum customers and critical severity.           │
│                                                                                                                 │
│  Using Tool: Ticketing_System_Interface                                                                         │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── Tool Input ───────────────────────────────────────────────────╮
│                                                                                                                 │
│  {                                                                                                              │
│    "action": "read_queue",                                                                                      │
│    "ticket_id": null,                                                                                           │
│    "new_status": null,                                                                                          │
│    "agent_notes": null                                                                                          │
│  }                                                                                                              │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── Tool Output ──────────────────────────────────────────────────╮
│                                                                                                                 │
│  Ticket ID: TIC-GEN-001 | Priority: medium | SLA: gold | Subject: General system query                          │
│  Ticket ID: TIC-IAM-002 | Priority: critical | SLA: platinum | Subject: URGENT: Cannot approve wires            │
│  Ticket ID: TIC-HW-105 | Priority: high | SLA: gold | Subject: Blue Screen of Death                             │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

...

╭──────────────────────────────────────────── 🔧 Agent Tool Execution ────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Technical Issue Analyst                                                                                 │
│                                                                                                                 │
│  Thought: Thought: The knowledge base suggests common resolutions include syncing time on the authenticator     │
│  app and network toggling to ensure the push notification reaches the device. Since the ticket does not         │
│  indicate these steps have been attempted, I will update the ticket with these recommendations as a resolution  │
│  and mark it as resolved assuming the user can follow those instructions.                                       │
│                                                                                                                 │
│  Using Tool: Ticketing_System_Interface                                                                         │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── Tool Input ───────────────────────────────────────────────────╮
│                                                                                                                 │
│  {                                                                                                              │
│    "action": "update_status",                                                                                   │
│    "ticket_id": "TIC-IAM-002",                                                                                  │
│    "new_status": "resolved",                                                                                    │
│    "agent_notes": "Resolved by advising user to sync time on authenticator app and toggle network connection    │
│  to fix token push notification issue in ERP system."                                                           │
│  }                                                                                                              │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── Tool Output ──────────────────────────────────────────────────╮
│                                                                                                                 │
│  Ticket TIC-IAM-002 updated successfully.                                                                       │
│                                                                                                                 │
│                                                                                                                 │
│  You ONLY have access to the following tools, and should NEVER make up tools that are not listed here:          │
│                                                                                                                 │
│  Tool Name: Ticketing_System_Interface                                                                          │
│  Tool Arguments: {                                                                                              │
│    "properties": {                                                                                              │
│      "action": {                                                                                                │
│        "description": "Action to perform: 'read_queue', 'get_ticket', or 'update_status'.",                     │
│        "title": "Action",                                                                                       │
│        "type": "string"                                                                                         │
│      },                                                                                                         │
│      "ticket_id": {                                                                                             │
│        "anyOf": [                                                                                               │
│          {                                                                                                      │
│            "type": "string"                                                                                     │
│          },                                                                                                     │
│          {                                                                                                      │
│            "type": "null"                                                                                       │
│          }                                                                                                      │
│        ],                                                                                                       │
│        "default": null,                                                                                         │
│        "description": "The ID of the ticket (e.g., TIC-IAM-002). Required for details/updates.",                │
│        "title": "Ticket Id"                                                                                     │
│      },                                                                                                         │
│      "new_status": {                                                                                            │
│        "anyOf": [                                                                                               │
│          {                                                                                                      │
│            "type": "string"                                                                                     │
│          },                                                                                                     │
│          {                                                                                                      │
│            "type": "null"                             

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Technical Issue Analyst                                                                                 │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  Ticket TIC-IAM-002 has been resolved. The resolution involved advising the user to sync time on the            │
│  authenticator app and toggle network connection to fix the token push notification issue in the ERP system.    │
│  The ticket status was updated to 'resolved' with these resolution notes.                                       │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯



########################
## Final Result ##
########################

Ticket TIC-IAM-002 has been resolved. The resolution involved advising the user to sync time on the authenticator app and toggle network connection to fix the token push notification issue in the ERP system. The ticket status was updated to 'resolved' with these resolution notes.


